In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"]="expandable_segments:True"
import argparse
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import gzip
from Bio import SeqIO
import math
from sklearn.model_selection import train_test_split
from create_embeddings import create_embeddings

importing torch
importing evo2
importing partial
importing hugging face


/home/jong505/conda/miniconda3/envs/evo2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


importing pkgutil
importing torch
importing typing
importing yaml
importing vortex1
importing vortex2
importing vortex3
importing vortex4
importing evo2 scoring
importing evo2 util


In [ ]:
def create_train_df(merged: pd.DataFrame, refseq: list):
    #output dict.
    output = {}
    # loop trough reference sequences.
    for file in refseq:
        #open refseq 
        with gzip.open(file, "rt") as f:
            # loop trough files in refseq (wich are mostly chromosones).
            for record in tqdm(SeqIO.parse(f, format="fasta"), desc=f"Collecting sequences from{file}"):
                # get the name of the current name of the refseq (Wich genome we are looking at).
                file_name = file.stem.split(".")[0]
                # get the name of the current file in the refseq (Wich chromosone we are looking at).
                chr_name = record.id.split(".")[0]
                # filter the df such that we are only looking at rows that have a segment that is from the current genome and chromosone.
                # We are doing this to reduce the size of the forloop, to improve speed.
                df_focus = merged[(merged[["genome_x", "genome_y"]] == file_name).any(axis=1) & (merged[["list_x", "list_y"]] == chr_name).any(axis=1)]
                # loop over df
                for indx, r in df_focus.iterrows():
                    # for the current row see if genome_x and or genome_y is the genome that is opened right now.
                    genome = r[r==file_name]
                    # loop over genome_x or genome_y or both debending on what genome is open right now.
                    for i, _ in genome.items():
                        # set x_y to either x or y depending on wich genome we are looking at
                        x_y = i.split("_")[1]
                        # check if the current (iadh) segment is from the correct chromosone that we have opend right now.
                        if r[f"list_{x_y}"] == chr_name:
                            # see if a dict already exists for the current id(indx), if so we update it, if not we create the id and write the inital data.
                            output.setdefault(r["id"], {}).update({f"genome_{x_y}": r[f"genome_{x_y}"],
                                                                f"chr_{x_y}": r[f"list_{x_y}"],
                                                                f"segment_id_{x_y}": r[f"segment_id_{x_y}"],
                                                                f"len_profile_{x_y}": r[f"len_profile_{x_y}"],
                                                                f"seq_{x_y}": str(record.seq[r[f"start_{x_y}"]-1:r[f"stop_{x_y}"]])})
                            
    return pd.DataFrame.from_dict(output, orient='index')[["genome_x", "chr_x", "len_profile_x", "segment_id_x", "genome_y", "chr_y", "len_profile_y", "segment_id_y", "seq_x", "seq_y"]]

def genes_to_fam(df, gene_fam):
    # create series to map gene to fam
    gene_to_fam = gene_fam.set_index("gene")["fam"]
    df_x = (
        df[["genes_x"]]
        # instead of having i1: [g1, g2 etc] we go to
        # i1: g1
        # i1: g2
        .explode("genes_x")
        # add column fam for wich we map gene_x using gene to fam
        .assign(fam=lambda df: df["genes_x"].map(gene_to_fam))
        # undo the explode
        .groupby(level=0)["fam"]
        # turn every group by section into list
        .agg(list)
        # rename colum fam
        .rename("fams_x")
    )

    df_y = (
        df[["genes_y"]]
        .explode("genes_y")
        .assign(fam=lambda df: df["genes_y"].map(gene_to_fam))
        .groupby(level=0)["fam"]
        .agg(list)
        .rename("fams_y")
    )
    return df.join([df_x, df_y])

def create_test_df_og(train_df: pd.DataFrame):
    df_c = train_df.copy() 
    df_c["og_index"] = df_c.index

    grouped = df_c.groupby(["len_profile_x", "len_profile_y"])
    shuffled_parts = []

    for _, group in grouped:
        # makes sure group is not to small to shuffel
        if len(group) == 1:
            shuffled_parts.append(group)
            print("Error group to small !")
            continue
        
        print(group["seq_y"].duplicated().sum(),"-",(group.shape[0]/2))

        if group["seq_y"].duplicated(keep=False).sum() > (group.shape[0]/2):
            print("Error group to similar !")
            continue

        shuffled_group = group.copy()
        same = True
        # shuffel everything till there are no same combinations as before


        while same:
            shuffled_values = shuffled_group[["genome_y", "chr_y", "len_profile_y", "segment_id_y", "seq_y"]].sample(frac=1, random_state=None)
            shuffled_group[["genome_y", "chr_y", "len_profile_y","segment_id_y", "seq_y"]] = shuffled_values[["genome_y", "chr_y", "len_profile_y","segment_id_y", "seq_y"]].values
            
            if shuffled_group[shuffled_group["seq_y"] == group["seq_y"]]["seq_y"].notna().sum() == 0:
                same = False
            
        shuffled_parts.append(shuffled_group)
    return pd.concat(shuffled_parts).sort_values("og_index").drop(columns="og_index")

def create_train_test_val(df):
    x_index, x_val_index, _, _ = train_test_split(df.index, df.similar.astype(int).values, test_size=0.30, random_state=42)
    df_val = df.iloc[x_val_index]
    df = df.iloc[x_index].reset_index(drop=True)
    x_train_index, x_test_index, _, _ = train_test_split(df.index, df.similar.astype(int).values, test_size=0.20, random_state=42)
    return df.iloc[x_train_index], df.iloc[x_test_index], df_val

def filter_df(merged, seg_len):
    df = pd.read_csv(merged, sep="\t", header=0)
    df = df[(df["len_profile_x"]<=seg_len) & (df["len_profile_y"]<=seg_len) & (df["genome_x"] != df["genome_y"])]
    return df 


In [5]:
list_elements_path = Path("/home/jong505/thesis/iadh/iadh_out/aar_ath_bol_chi_cpa_tha/list_elements.txt")
# list_elements = pd.read_csv(list_elements, sep='\t', header=0)

merged_path = Path("/home/jong505/thesis/iadh/iadh_out/aar_ath_bol_chi_cpa_tha/merged_results.tsv")
seg_len = 7
# merged = pd.read_csv(merged_path, sep="\t", header=0)
# merged = merged[(merged["len_profile_x"]<=seg_len) & (merged["len_profile_y"]<=seg_len) & (merged["genome_x"] != merged["genome_y"])]


data_p = Path("/home/jong505/thesis/iadh/data")
refseqs = [f"{data_p}/ath.fasta.gz", f"{data_p}/aar.fasta.gz", f"{data_p}/bol.fasta.gz", f"{data_p}/tha.fasta.gz", f"{data_p}/chi.fasta.gz", f"{data_p}/cpa.fasta.gz"]
refseqs = [Path(f) for f in refseqs]

gene_fam_path = Path("/home/jong505/thesis/iadh/data/gene_fam_parsed.tsv")
# gene_fam = pd.read_csv(gene_fam_path, sep="\t", header=None,  names=['gene', 'fam'])

output_prefix = Path("data/aar_ath_bol_chi_cpa_tha/sm7_50000_new_neg")
output_prefix_seq = Path("data/aar_ath_bol_chi_cpa_tha/sm7_50000_new_neg")
len_nuc = 50000

In [6]:
filtered_df = filter_df(merged_path, seg_len)

# Only put samples in there that have the same orientation. 
# filtered_df = filtered_df[ (filtered_df["sim_orientations_x"] >= 1) & (filtered_df["sim_orientations_y"] >= 1)]

train_df = create_train_df(filtered_df, refseqs)

if len_nuc != 0:
    print(f"Filtering for max length of {len_nuc} nucliotides")
    before = train_df.shape[0]
    train_df = train_df[train_df.apply(lambda r: max(len(r['seq_x']), len(r['seq_y'])) < len_nuc, axis=1)]
    print(f"Went fom {before} to {train_df.shape[0]} pairs")

# print("Creating negative samples")
# test_df = create_test_df(train_df)


Filtering for max length of 50000 nucliotides
Went fom 1545 to 1323 pairs


In [60]:
def create_negative_samples(df, gene_fam_path, list_elements):
    gene_fam = pd.read_csv(gene_fam_path, sep="\t", header=None,  names=['gene', 'fam'])
    list_elements = pd.read_csv(list_elements, sep='\t', header=0)
    # create series that can be used to map segment to gene
    segment_to_gene = list_elements.set_index("segment")["gene"]
    # aggregate from:
    # seg1: g1
    # seg1: g2
    # to: seg1: [g1, g2]
    segment_to_gene = segment_to_gene.groupby("segment").agg(list)
    # add collumn named gene_x in wich we use the segment id to look up what genes are in the segment using segment_to_gene
    df = df.assign(genes_x=lambda df: df["segment_id_x"].map(segment_to_gene))
    df = df.assign(genes_y=lambda df: df["segment_id_y"].map(segment_to_gene))
    df = genes_to_fam(df, gene_fam)

    # df_c = df.copy()
    same = True

    shuffled_df = df.copy().reset_index(drop=True)
    # display(shuffled_df.head())
    limit = 5000
    counter = 0
    lowest = 99999999999999
    best = ""
    while same:
        if counter>= limit:
            print("exeeded limt, failed to find new shuffel")
            same=False
        counter+=1
        
        shuffled_df[["genome_y", "chr_y", "len_profile_y", "segment_id_y", "seq_y", "fams_y"]] = df[["genome_y", "chr_y", "len_profile_y", "segment_id_y", "seq_y", "fams_y"]].sample(frac=1, random_state=None).reset_index(drop=True)
        
        shuffled_df["sim"] = [bool(set(x) & set(y)) for x, y in zip(shuffled_df["fams_x"], shuffled_df["fams_y"])]
        if shuffled_df.sim.sum()< lowest:
            best = shuffled_df.copy()
            lowest = shuffled_df.sim.sum()

        if True not in [bool(set(x) & set(y)) for x, y in zip(df["fams_x"], shuffled_df["fams_y"])]:
            return shuffled_df
        
        # shuffled_df[shuffled_df.sim][["genome_y", "chr_y", "len_profile_y", "segment_id_y", "seq_y", "fams_y"]] = shuffled_df[shuffled_df.sim][["genome_y", "chr_y", "len_profile_y", "segment_id_y", "seq_y", "fams_y"]].sample(frac=1, random_state=None)

    print(f"dropping {best.sim.sum()} samples because of sim")
    return best[~best.sim]

def get_from_dif_fam(fams, pool: pd.DataFrame, limit=1000):
    for i in range(1000):
        new = pool.sample(n=1)
        if bool(set(new["fams_y"]) & set(fams)):
            pool.drop(index=new.index, inplace=True)
            return new, pool
    print("No new sample was found")
    return None, pool

In [61]:
test_df = create_negative_samples(train_df, gene_fam_path, list_elements_path)
test_df["sim"] = [bool(set(x) & set(y)) for x, y in zip(test_df["fams_x"], test_df["fams_y"])]

exeeded limt, failed to find new shuffel
dropping 2 samples because of sim


In [ ]:
train_df["multiplicon_id"] = pd.to_numeric(train_df.index, downcast='integer')
test_df["multiplicon_id"] = pd.NA
train_df["similar"] = True
test_df["similar"] = False
df = pd.concat([train_df, test_df], ignore_index=True)
# print(f"Writing output to {output_prefix}")
# df[["segment_id", "similar", "genome_x", "chr_x", "len_profile_x", "genome_y", "chr_y", "len_profile_y", "seq_x", "seq_y"]].to_csv(output_path, sep='\t')

train, test, val = create_train_test_val(df)
print(f"shapes: train:{train.shape}, train:{test.shape}, train:{val.shape}")

train.drop(columns=["seq_x", "seq_y"]).to_csv(str(output_prefix)+"_train.tsv", sep="\t")
test.drop(columns=["seq_x", "seq_y"]).to_csv(str(output_prefix)+"_test.tsv", sep="\t")
val.drop(columns=["seq_x", "seq_y"]).to_csv(str(output_prefix)+"_val.tsv", sep="\t")


if output_prefix_seq != "":
    train.to_csv(str(output_prefix_seq)+"_train_seq.tsv", sep="\t")
    test.to_csv(str(output_prefix_seq)+"_test_seq.tsv", sep="\t")
    val.to_csv(str(output_prefix_seq)+"_val_seq.tsv", sep="\t")

print("Generating embeddings")
embeddings = create_embeddings(df)
embeddings.to_csv(str(output_prefix_seq)+"_embeddings.tsv", sep="\t")

# print(f"generating train embeddings")
# train_em = create_embeddings(train)
# print(train_em.head())
# print(str(output_prefix)+"_train.tsv")
# train_em.to_csv(str(output_prefix)+"_train.tsv", sep="\t")

# print(f"generating test embeddings")
# test_em = create_embeddings(test)
# test_em.to_csv(str(output_prefix)+"_test.tsv", sep="\t")

# print(f"generating val embeddings ")
# val_em = create_embeddings(val)
# val_em.to_csv(str(output_prefix)+"_val.tsv", sep="\t")